![](https://raw.githubusercontent.com/stack-of-tasks/pinocchio/master/doc/images/pinocchio-logo-large.png)

# Pinocchio: rigib-body algorithms


In [1]:
import magic_donotload

NB: as for all the tutorials, a magic command %do_not_load is introduced to hide the solutions to some questions. Change it for %load if you want to see (and execute) the solution.


## Set up
We will need Pinocchio, the robot models stored in the package *example-robot-data*, a viewer (either GepettoViewer or MeshCat), some basic linear-algebra operators and the SciPy optimizers.

In [2]:
import pinocchio as pin
import example_robot_data as robex
import numpy as np
from numpy.linalg import inv,pinv,eig,norm,svd,det
from scipy.optimize import fmin_bfgs
import time
import copy

## 1. Load and display the robot

Pinocchio is a library to compute different quantities related to the robot model, like body positions, inertias, gravity or dynamic effects, joint jacobians, etc. For that, we need first to define the robot model. The easiest solution is to load it from a [URDF model](https://ocw.tudelft.nl/course-lectures/2-2-1-introduction-to-urdf/). This can be done with the function **buildModelFromUrdf**.

The package **example-robot-data** proposes to directyl load several URDF models with a single line of code. This is what we are going to use for the tutorial. Look at the code inside the package if you want to adapt it to the URDF model of your favorite robot.


In [3]:
robot = robex.loadTalosArm()   # Load a 6-dof manipulator arm

The robot can be visualized in a viewer. Two viewers are proposed by default in Pinocchio: Gepetto-Viewer and MeshCat. Both display the robot model in an external windows managed by another processus.
- MeshCat is browser based, which makes it very convenient in the jupyter context. 
- Gepetto-Viewer is the historical viewer for Gepetto, and is more powerful. But you need to first run the command **gepetto-viewer** in a terminal, before starting your python command.

In [4]:
#Viewer = pin.visualize.GepettoVisualizer
Viewer = pin.visualize.MeshcatVisualizer

viz = Viewer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(loadModel=True)
viz.display(robot.q0)


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


If chosing MeshCat, you can open the display directly inside the notebook.

In [5]:
viz.viewer.jupyter_cell()

## 2. Pinocchio's philosophy (model, data and algorithms)

### Model vs data

Pinocchio is not extensively using the object oriented design pattern. We rather regroup the informations and buffers in some important data structure, then access to these structures using algorithms written in static functions.

The two main data structures are `robot.model` and `robot.data`.

In [6]:
rmodel = robot.model
rdata = rmodel.createData()

The `robot.model` structure contains all the *constant* informations that the algorithms need to evaluate. It is typically loaded from a file describing the model (URDF). The `robot.model` is typically constant and is not modified by algorithms.

The `robot.data` structure contains the memory buffers that the algorithms needs to store intermediary values or the final results to return to the user. 

### Joints names and indexes

You can get the list of the joint names with the following:

In [7]:
for n in rmodel.names: print(n)

universe
arm_left_2_joint
arm_left_3_joint
arm_left_4_joint
arm_left_5_joint
arm_left_6_joint
arm_left_7_joint
gripper_left_joint


In what follows, we will specifically use the joint named `gripper_left_joint` whose index can be obtained with:

In [8]:
jointIndex = rmodel.getJointId("gripper_left_joint")

### A first algorithm: random configuration

For example, you can pick a random configuration by calling the algorithm `randomConfiguration`. This algorithm just needs the `robot.model` (as no intermediary buffer are needed).

In [9]:
q = pin.randomConfiguration(rmodel)

### A second algorithm: forward kinematics

Another example is the algorithm to compute the foward kinematics. It recursively computes the placement of all the joint frames of the kinematic tree, and stores the results in `robot.data.oMj`, which is an array indexed by the joint indexes.

In [10]:
pin.forwardKinematics(rmodel,rdata,q)
rdata.oMi[jointIndex]

  R =
 -0.158898  -0.982742 -0.0947115
 -0.652084   0.176493  -0.737318
  0.741309 -0.0553985  -0.668874
  p = 0.179678 0.565002 0.216257

***
![recap](recap.png)
### Recap of the main syntaxts exposed in this section
- `robot.model` and `robot.data` structures
- `pin.randomConfiguration(robot.model)` to sample a random configuration
- `pin.forwardKinematics(robot.model,robot.data,q)` to compute joint frame placements.
- `robot.data.oMi[jointIdex]` to access a particular joint frame.
***

## 3. 3d cost: optimizing the end effector position

We will now define a first cost function, that penalizes the distance between the robot gripper and a target.
For that, let's define a target position `ptarget` of dimension 3. 

In [11]:
ptarget = np.array([.5,.1,.3])

### Joints and frames
As explained upper, the position of the joint `jointIndex` is stored in `robot.data.oMi[jointIndex].translation`, and is recomputed by `pin.forwardKinematics`. 

In Pinocchio, each joint is defined by its joint frame, whose **placement**, i.e. position+orientation, is stored in `robot.data.oMi`. 

We also defined in addition other *operational* frames. They are defined by a fixed placement with respect to their parent joint frame. Denoting by oMi the placement of the parent joint frame (function of q), and by iMf the fixed placement of the operational frame with respect to the parent joint frame, the placement of the operational frame with respect to the world is easily computed by oMf(q) = oMi(q)iMf. 

A complete list of available frames is stored in `robot.frames`. 


In [12]:
for f in robot.model.frames: print(f.name)

universe
root_joint
arm_left_1_link
arm_left_2_joint
arm_left_2_link
arm_left_3_joint
arm_left_3_link
arm_left_4_joint
arm_left_4_link
arm_left_5_joint
arm_left_5_link
arm_left_6_joint
arm_left_6_link
arm_left_7_joint
arm_left_7_link
wrist_left_ft_joint
wrist_left_ft_link
wrist_left_tool_joint
wrist_left_ft_tool_link
gripper_left_base_link_joint
gripper_left_base_link
gripper_left_inner_double_joint
gripper_left_inner_double_link
gripper_left_fingertip_1_joint
gripper_left_fingertip_1_link
gripper_left_fingertip_2_joint
gripper_left_fingertip_2_link
gripper_left_inner_single_joint
gripper_left_inner_single_link
gripper_left_fingertip_3_joint
gripper_left_fingertip_3_link
gripper_left_joint
gripper_left_motor_double_link
gripper_left_motor_single_joint
gripper_left_motor_single_link


All frame placements are computed directly by calling `pin.framesForwardKinematics` (or alternatively by calling `pin.updateFramePlacement` or `pin.updateFramePlacements` after a call to `pin.forwardKinematics`).

In [13]:
pin.framesForwardKinematics(rmodel,rdata,q)

For the tutorial, we will use the frame `gripper_left_fingertip_1_link`:

In [14]:
frameIndex = rmodel.getFrameId('gripper_left_fingertip_1_link')

### Cost-function template
In this tutorial, the cost functions will all be defined following the same template:

In [15]:
class Cost:
    def __init__(self,rmodel,rdata,viz=None):  # add any other arguments you like
        self.rmodel = rmodel
        self.rdata  = rdata
        self.viz = viz
    def calc(self,q):
        ### Add the code to recompute your cost here
        cost = 0
        return cost
    def callback(self,q):
        if viz is None: return
        # Display something in viz ...

We will see later that the callback can be used to display or print data during the optimization. For example, we may want to display the robot configuration, and visualize the target position with a simple sphere added to the viewer.

TODO Guilhem: add a simple sphere to MeshCat

In [16]:
try:
    viz.addBox("world/box",  .05,.1,.2, [1., .2, .2, .5])
    viz.addSphere("world/ball", .05, [.2, .2, 1., .5])
    viz.applyConfiguration('world/box', [0.5,.2,.2,1,0,0,0])
    viz.applyConfiguration('world/ball', [-0.5,.2,.2,1,0,0,0])
except:
    print("This code is only valid for Gepetto-Viewer ... TODO for MeshCat")

This code is only valid for Gepetto-Viewer ... TODO for MeshCat


![Question](question.png) 
### Question
Implement a `Cost` class computing the quadratic cost `p(q)-ptarget`.

In [34]:
# %load -r 6-31 costs.py
class Cost3d:
    def __init__(self,rmodel,rdata,frameIndex=None, ptarget=None,viz=None):
        self.rmodel = rmodel
        self.rdata = rdata
        self.ptarget = ptarget if ptarget is not None \
                       else  np.array([0.5, 0.1, 0.27])
        self.frameIndex = frameIndex if frameIndex is not None else robot.model.nframes-1
        self.viz = viz
        
    def residual(self,q):
        pin.framesForwardKinematics(self.rmodel,self.rdata,q)
        M = self.rdata.oMf[self.frameIndex]
        return (M.translation-self.ptarget)

    def calc(self,q):
        return sum(self.residual(q)**2)

    ### --- Callback
    def callback(self,q):
        if self.viz is None: return
        pin.framesForwardKinematics(self.model,self.rdata,q)
        M = self.rdata.oMf[self.frameIndex]
        self.viz.applyConfiguration('world/blue', pin.SE3ToXYZQUATtuple(M))
        self.viz.applyConfiguration('world/box', self.ptarget.tolist()+[1,0,0,0])
        self.viz.display(q)
        time.sleep(1e-2)

![RECAP](recap.png)
### Recap of the main syntaxts exposed in this section
- Frame placements are computed by `pin.framesForwardKinematics(rmodel,rdata,q)`, and accessed by `rdata.oMf[frameIndex]`.
- Frame translation is a 3d array stored in `rdata.oMf[frameIndex].translation`.



## 4. 6d cost: optimizing the end effector placement (position+orientation)

We will now define a cost function that penalizes the placement, i.e. rotation+translation between an operational frame and a target frame.

### Spatial algebra in Pinocchio
Most of the physical quantifies stored in Pinocchio are 6D quantities, that are element of the so-called *Spatial Algebra*, following Roy Featherstone definition and namings. Featherstone's work, and in particular [his excellent 2008 book](https://link.springer.com/content/pdf/10.1007%2F978-1-4899-7560-7.pdf) is the basis for all the algorithms in Pinocchio.

Frame placement, formally elements of the Lie group SE(3), are represented in Pinocchio by an object of class `pin.SE3` containing a 3x3 rotation array and and 3 translation vector. Placements can by multiplied and inverted.

In [18]:
M1 = pin.SE3.Identity()
M2 = pin.SE3.Random()
print(M2,M2.rotation, M2.translation)
M3 = M1*M2
M4 = M2.inverse()

  R =
  -0.434594    0.194442   -0.879386
   -0.90059   -0.102588     0.42239
-0.00808388    0.975535    0.219697
  p = -0.967399 -0.514226 -0.725537
 [[-0.43459386  0.19444206 -0.87938641]
 [-0.90059027 -0.10258764  0.42238957]
 [-0.00808388  0.97553476  0.21969659]] [-0.96739886 -0.51422646 -0.72553685]


SE(3) comes with `log` and `exp` operations. The `log` maps a SE3 placement into the 6D velocity that should be applied during 1 second to obtain this placement, starting from identity. In Pinocchio, 6D velocity are represented as `pin.Motion` object, and can be mapped to 6D array.

In [21]:
nu = pin.log(M1)
print(nu.vector)

[0. 0. 0. 0. 0. 0.]


We will not need much of spatial algebra in this tutorial. See the class `pin.Motion`, `pin.Force`, `pin.Inertia` for more details. 

### Distances between frames
The `log` operator can be used to define a distance between placements. The norm of the log is evidently positive and null only in the placement is identity. Consequently, `log(M1.inverse()*M2)` is a positive scalar that is null only if `M1==M2`.

![Question](question.png)
Following the same previous pattern, define a Cost6d class penalizing the distance between a frame attached to the robot and a reference fixed frame.

In [24]:
# %load -r 42-67 costs.py
class Cost6d:
    def __init__(self,rmodel,rdata,frameIndex=None, Mtarget=None,viz=None):
        self.rmodel = rmodel
        self.rdata = rdata
        self.Mtarget = Mtarget if Mtarget is not None \
                       else pin.SE3(pin.utils.rotate('x',3.14/4), np.array([0.5, 0.1, 0.27]))  # x,y,z
        self.frameIndex = frameIndex if frameIndex is not None else robot.model.nframes-1
        self.viz = viz

    def residual(self,q):
        '''Compute score from a configuration'''
        pin.forwardKinematics(self.rmodel,self.rdata,q)
        M = pin.updateFramePlacement(self.rmodel,self.rdata,self.frameIndex)
        self.deltaM = self.Mtarget.inverse() * M
        return pin.log(self.deltaM).vector

    def calc(self,q):
        return sum(self.residual(q)**2)

    def callback(self,q):
        if self.viz is None: return
        self.viz.applyConfiguration('world/blue', pin.SE3ToXYZQUATtuple(M))
        self.viz.applyConfiguration('world/box', pin.SE3ToXYZQUATtuple(self.Mtarget))
        self.viz.display(q)
        time.sleep(1e-2)


![RECAP](recap.png)
### Recap of the main syntaxts exposed in this section
- SE3 logarithm is implemented by `pin.log`, returns a `pin.Motion` class that can be converted into a vector with `pin.log(M).vector`.

## 5. Redundancy and introduction of a posture cost

We will now run the optimizer with the two cost functions defined above, and add a posture cost to regularize the rank-deficient hessian.

### Runing the optimizer
We will use the optimizer BFGS from SciPy. It is quasi-newton algorithm, which means that it only needs the first-order derivatives (while having super-linear convergence). Even better, it automatically approximates the derivatives by finite differences if you don't provide them. 


In [25]:
viz.viewer.jupyter_cell()

In [35]:
cost = Cost3d(rmodel,rdata)
qguess = robot.q0.copy()
qopt = fmin_bfgs(cost.calc,qguess,callback=cost.callback)
viz.display(qopt)

Optimization terminated successfully.
         Current function value: 0.007774
         Iterations: 45
         Function evaluations: 486
         Gradient evaluations: 54


### Redundancy

The arm of the robot Talos, that we used by default in the notebook, as 6 degrees of freedom (if not considering the gripper). If using the 3D cost function, there is a continuum of solutions, as the kinematic is redundant for achieve a pointing tasks. You can obtain different optimum by changing the initial guess. Each new run with a random initial guess gives you a new optimum.

In [45]:
qguess = pin.randomConfiguration(rmodel)
qopt = fmin_bfgs(cost.calc,qguess,callback=cost.callback)
viz.display(qopt)

Optimization terminated successfully.
         Current function value: 0.007774
         Iterations: 29
         Function evaluations: 306
         Gradient evaluations: 34


We will now add a small regularization to the cost, by optimizing a full-rank term on the robot posture, to make the solution unique indenpendantly from the robot kinematics and the considered task cost.

![Question](question.png)
Introduce a new cost function for penalizing the distance of the robot posture to a reference posture, for example `robot.q0`.

In [53]:
%do_not_load -r 75-88 costs.py

### Optimize a sum of cost

Now we can define an ad-hoc cost that makes a sum of both costs.

In [58]:
class SumOfCost:
    def __init__(self,costs,weights):
        self.costs = costs
        self.weights = np.array(weights)
    def calc(self,q):
        return sum(self.weights*[ cost.calc(q) for cost in self.costs] )
mycost = SumOfCost([Cost3d(rmodel,rdata),CostPosture(rmodel,rdata)],[1,1e-3])

And we optimize this new cost.

In [59]:
fmin_bfgs(mycost.calc,qguess)

Optimization terminated successfully.
         Current function value: 0.017505
         Iterations: 59
         Function evaluations: 576
         Gradient evaluations: 64


array([ 3.04665587,  0.09828725, -1.74024907, -1.27813069,  0.16829691,
        0.05891853, -0.35917417])

![RECAP](recap.png)
### Recap of the main syntaxts exposed in this section
- The BFGS solver is called by `fmin_bfgs(cost.calc,qguess,cost.callback)`. 



## 6. Gravity cost: introducing the dynamic model

The posture cost is a nice regularization but requires a reference posture that may be hard to define. We will now define a cost to minimize the gravity torque, as an alternative regularization.

### Dynamics in Pinocchio

The whole-body dynamics equation can be written as:
$$M(q) a_q + b(q,v_q) = \tau_q   $$
where $q,v_q,a_q$ are the position, velocity and acceleration in the configuration space, $M(q)$ is the configuration-space inertia matrix, of size `robot.model.nv`x`robot.model.nv`, $b(q,v_q)$ gathers all the drift terms (Coriolis, centrifugal, gravity) and $\tau_q$ are the joint torques. We write $v_q$ and $a_q$ because in general $q$ (of size `robot.model.nq`) does not have the same size as its derivatives, although they corresponds to $\dot{q}$ and $\ddot{q}$ for now.

### Computing the gravity term
This equation corresponds to the inverse dynamics. We can evaluate parts of it or the entire equation, as we will see next. Let's start with a simple case. 

The gravity term corresponds to the torques when the robot has no velocity nor acceleration:
$$g(q) = b(q,v_q=0) = dyninv(q,v_q=0,a_q=0)$$
In Pinocchio, it can be directly computed with:

In [61]:
g = pin.computeGeneralizedGravity(rmodel,rdata,q)

![question](question.png)
Define a new cost function that compute the squared norm of the gravity.

In [85]:
# %load -r 99-107 costs.py
class CostGravity:
    def __init__(self,rmodel,rdata,viz=None):
        self.rmodel = rmodel
        self.rdata = rdata
        self.viz = viz
    def residual(self,q):
        return pin.computeGeneralizedGravity(self.rmodel,self.rdata,q)
    def calc(self,q):
        return sum(self.residual(q)**2)

![RECAP](recap.png)
### Recap of the main syntaxts exposed in this section
- The gravity torque can be computed by `pin.computeGeneralizedGravity(rmodel,rdata,q)`.

## 7. Weighted gravity cost: RNEA and ABA recursive algorithms

Minimizing the gravity cost is often obtained by bringing the robot to singular configurations. A better behavior might be obtained by a variation of this cost implying the generalized inertia matrix.

### Recursive algorithms in Pinocchio
The 3 most efficient algorithms to evaluate the dynamic equations are implemented in Pinocchio:
- the recursive Newton-Euler algorithm (RNEA) computes the inverse dynamics $\tau_q = invdyn(q,v_q,a_q)$
- the articulated rigid body algorithm (ABA) computes the direct dynamics $a_q = dirdyn(q,v_q,_tau_q)$
- the composite rigid body algorithm (CRBA) computes the mass matrix.
The 3 algorithms can be directly called by their names in Pinocchio.

In [75]:
vq = np.random.rand(rmodel.nv)*2-1
aq = np.random.rand(rmodel.nv)*2-1
tauq = pin.rnea(rmodel,rdata,q,vq,aq)
aq2 = pin.aba(rmodel,rdata,q,vq,tauq)
assert(norm(aq-aq2)<1e-6)
M = pin.crba(rmodel,rdata,q)

The gravity torque $g(q)=b(q,0)$ and the dynamic drift $b(q,vq)$ can be also computed by a truncated RNEA implementation.

In [74]:
b = pin.nle(rmodel,rdata,q,vq)
assert(norm(M@aq+b-tauq)<1e-6)

### Weighted gravity norm
A better posture regularization can be obtained by taking the norm of the gravity weighted by the inertia:
$$ l(q) = g(q)^T M(q)^{-1} g(q)$$

We can directly evaluate this expression by compute and inverting $M$. Yet it is more efficient to recognize that $M(q)^{-1} g(q) = -dirdyn(q,0,0)$ and can be evaluated by:

In [78]:
aq0 = -pin.aba(rmodel,rdata,q,np.zeros(rmodel.nv),np.zeros(rmodel.nv))

![question](question.png)
Implement a new cost function for this weighted gravity, and run it as the regularization of a reaching task.

In [87]:
# %load -r 114-126 costs.py
class CostWeightedGravity:
    """
    return g.T*inv(M)*g = g(q).T * aba(q,0,0) = rnea(q,0,0).T*aba(q,0,0)
    """
    def __init__(self,rmodel,rdata,viz=None):
        self.rmodel = rmodel
        self.rdata = rdata
        self.viz = viz
        self.v0 = np.zeros(self.rmodel.nv)  # for convenience in the evaluation
    def calc(self,q):
        g = pin.computeGeneralizedGravity(self.rmodel,self.rdata,q)
        taugrav = -pin.aba(self.rmodel,self.rdata,q,self.v0,self.v0)
        return np.dot(taugrav,g)

![RECAP](recap.png)
### Recap of the main syntaxts exposed in this section
- Inverse dynamics: `tauq = pin.rnea(rmodel,rdata,q,vq,aq)`
- Direct dynamcis: `


## 8. Working with a free floating robot

![RECAP](recap.png)
Recap of the main syntaxts exposed in this section.
